<a href="https://colab.research.google.com/github/ars-akash/glucoma-detection-cnn/blob/main/XGBC_RIM_ONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLassification using XGB

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
import keras 
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras import backend as K
from matplotlib import pyplot
from numpy import expand_dims

import tensorflow as tf

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [ ]:
!git clone https://github.com/seva100/optic-nerve-cnn.git

Cloning into 'optic-nerve-cnn'...
remote: Enumerating objects: 7634, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7634 (delta 0), reused 2 (delta 0), pack-reused 7631
Receiving objects: 100% (7634/7634), 706.89 MiB | 32.77 MiB/s, done.
Resolving deltas: 100% (3044/3044), done.
Checking out files: 100% (7340/7340), done.


In [ ]:
try:
  os.mkdir('/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages')
  os.mkdir('/content/optic-nerve-cnn/data/RIM-ONE v3/Healthy/testimages')
except:
  print('already exists')

After the above code execution, we have to add some test images to the folder manually. In the best case 10 images are taken to the test set. 

In [ ]:
path1 = '/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/'
path2 = '/content/optic-nerve-cnn/data/RIM-ONE v3/Healthy/Stereo Images/'
size = 256
train_images=[]
train_labels = []
for image in os.listdir(path1):  
  print(os.path.join(path1, image))
  label = 'Glaucoma and suspects'
  if ('.jpg' in os.path.join(path1, image)):
    x_img = cv2.imread(os.path.join(path1, image), cv2.IMREAD_COLOR)  
    try:
      x_img = x_img[0:1424, 0:1072]
      x_img = cv2.resize(x_img, (256, 256))
      
    except:
      x_img = x_img[0:1424, 1072:]
      x_img = cv2.resize(x_img, (256, 256))
    
    x_img = cv2.cvtColor(x_img, cv2.COLOR_RGB2BGR)
    train_images.append(x_img)
    train_labels.append(label)
  else:
    print('skipping')

for image in os.listdir(path2):  
  print(os.path.join(path2, image))
  label = 'healthy'
  if ('.jpg' in os.path.join(path2, image)):
    x_img = cv2.imread(os.path.join(path2, image), cv2.IMREAD_COLOR)    
    try:
      x_img = x_img[0:1424, 0:1072]
      x_img = cv2.resize(x_img, (256, 256))
    except:
      x_img = x_img[0:1424, 1072:]
      x_img = cv2.resize(x_img, (256, 256))

    x_img = cv2.cvtColor(x_img, cv2.COLOR_RGB2BGR)
    train_images.append(x_img)
    train_labels.append(label)
  else:
    print('skipping')

train_images = np.array(train_images)
train_labels = np.array(train_labels)

/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/G-17-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/G-29-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/S-34-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/S-32-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/S-7-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/G-31-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/G-20-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/G-33-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/S-17-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/G-39-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/Stereo Images/S-1-L.jpg
/content/optic-nerve-cnn/data/RIM-

In [ ]:
path1 = '/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/'
path2 = '/content/optic-nerve-cnn/data/RIM-ONE v3/Healthy/testimages/'

size = 256
test_images=[]
test_labels = []
for image in os.listdir(path1):  
  print(os.path.join(path1, image))
  label = 'Glaucoma and suspects'
  if ('.jpg' in os.path.join(path1, image)):
    x_img = cv2.imread(os.path.join(path1, image), cv2.IMREAD_COLOR)  
    try:
      x_img = x_img[0:1424, 0:1072]
      x_img = cv2.resize(x_img, (256, 256))
      
    except:
      x_img = x_img[0:1424, 1072:]
      x_img = cv2.resize(x_img, (256, 256))
  else:
    print('skipping')

  x_img = cv2.cvtColor(x_img, cv2.COLOR_RGB2BGR)
  test_images.append(x_img)
  test_labels.append(label)

for image in os.listdir(path2):  
  print(os.path.join(path2, image))
  label = 'healthy'
  if ('.jpg' in os.path.join(path1, image)):
    x_img = cv2.imread(os.path.join(path2, image), cv2.IMREAD_COLOR)  
    try:
      x_img = x_img[0:1424, 0:1072]
      x_img = cv2.resize(x_img, (256, 256))
      
    except:
      x_img = x_img[0:1424, 1072:]
      x_img = cv2.resize(x_img, (256, 256))
  else:
    print('skipping')
    
  x_img = cv2.cvtColor(x_img, cv2.COLOR_RGB2BGR)    
  test_images.append(x_img)
  test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)


/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-27-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-9-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-18-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-2-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-26-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-31-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-33-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-3-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-29-R.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-5-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testimages/S-30-L.jpg
/content/optic-nerve-cnn/data/RIM-ONE v3/Glaucoma and suspects/testim

In [ ]:
from sklearn import preprocessing

lev_enc = preprocessing.LabelEncoder()

#label encoding for the training data
lev_enc.fit(train_labels)
train_labels_encoded = lev_enc.transform(train_labels)

#label encoding for the testing data
lev_enc.fit(test_labels)
test_labels_encoded = lev_enc.transform(test_labels)


In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded #formal splitting of the dataset

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0 #taking the pixels of the image to normalization

from keras.applications.vgg16 import VGG16 

VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(size, size, 3)) #here we are creating a vgg model on the purpose of feature extraction


58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in VGG_model.layers:
	layer.trainable = False  #we gave the training layer false, because we dont want to train the data in the VGG16, rather extract the features from the model
    
VGG_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
feature_extractor = VGG_model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)  

X_for_training = features # we will feed this X_for_training to the XGB classifier

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier() #loaded the model
model.fit(X_for_training, y_train) #fitted the model


X_test_feature = VGG_model.predict(x_test) #we prepare the test data the prev.
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
#Now predict using the trained RF model. 
prediction = model.predict(X_test_features)

#Inverse le transform to get original label back. 
prediction = lev_enc.inverse_transform(prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(test_labels, prediction)
score = accuracy_score(test_labels, prediction)

sns.heatmap(cm, annot=True)

n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0)
input_img_feature=VGG_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction = model.predict(input_img_features)[0] 
prediction = lev_enc.inverse_transform([prediction])
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])
print('The Accuracy: %.3f' % score)

In [ ]:
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_feature=VGG_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction = model.predict(input_img_features)[0] 
prediction = lev_enc.inverse_transform([prediction])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])
print('accuracy: ', score)